In [32]:
from pydantic import BaseModel, EmailStr, Field, validator
from typing import List, Optional

Задача 1. Базовые модели 
Создайте следующие Pydantic-модели: 
Book 
– title: str 
– author: str 
– year: int 
– available: bool 
User 
– name: str 
– email: str (с валидацией email) 
– membership_id: str

In [33]:
from pydantic import field_validator


class Book(BaseModel):
    title: str
    author: str
    year: int
    available: bool = True
    categories: List[str] = Field(default_factory=list)

    @field_validator('categories')
    def validate_categories(cls, value):
        if not value:
            raise ValueError("У книги должна быть хотя бы одна категория")
        return value


class User(BaseModel):
    name: str
    email: EmailStr
    membership_id: str

Задача 2. Функции с аннотациями типов 
Напишите следующие функции, используя аннотации типов:
add_book(...) -> ...
find_book(...) -> ...
is_book_borrow(...) -> ...
return_book(...) -> ...

In [34]:
def add_book(books: List[Book], book: Book) -> List[Book]:
    """Добавляет книгу в список библиотеки"""
    books.append(book)
    return books


def find_book(books: List[Book], title: str) -> Optional[Book]:
    """Ищет книгу по названию"""
    for book in books:
        if book.title.lower() == title.lower():
            return book
    return None


def is_book_borrow(book: Book) -> bool:
    """Проверяет, доступна ли книга для выдачи"""
    if not book.available:
        raise BookNotAvailable(f"Книга '{book.title}' недоступна для выдачи")
    book.available = False
    return True


def return_book(book: Book) -> bool:
    """Возвращает книгу в библиотеку"""
    book.available = True
    return True


Задача 3. Расширенная модель и валидация
Создайте модель Library: 
– books: … 
– users: …
Добавьте в модель Book поле categories: List[str] с валидацией.
Реализуйте метод total_books() -> ... для модели Library.

In [35]:
class Library(BaseModel):
    books: List[Book] = Field(default_factory=list)
    users: List[User] = Field(default_factory=list)

    def total_books(self) -> int:
        """Возвращает общее количество книг в библиотеке"""
        return len(self.books)


In [36]:
class BookNotAvailable(Exception):
    """Исключение при попытке взять недоступную книгу"""
    pass


In [37]:
# Пример данных
book1 = Book(title="1984", author="Джордж Оруэлл", year=1949, available=True, categories=["Роман", "Антиутопия"])
book2 = Book(title="Война и мир", author="Лев Толстой", year=1869, available=True, categories=["Классика"])

user1 = User(name="Даниил", email="danil@example.com", membership_id="U001")

library = Library(books=[book1, book2], users=[user1])

print("Всего книг в библиотеке:", library.total_books())

# Поиск книги
found = find_book(library.books, "1984")
print("Найдена книга:", found.title if found else "не найдена")

# Выдача книги
try:
    is_book_borrow(found)
    print(f"Книга '{found.title}' выдана.")
except BookNotAvailable as e:
    print("Ошибка:", e)

# Попытка выдать снова
try:
    is_book_borrow(found)
except BookNotAvailable as e:
    print("Ошибка:", e)

# Возврат книги
return_book(found)
print(f"Книга '{found.title}' возвращена. Доступна:", found.available)


Всего книг в библиотеке: 2
Найдена книга: 1984
Книга '1984' выдана.
Ошибка: Книга '1984' недоступна для выдачи
Книга '1984' возвращена. Доступна: True
